pytorch有很多自己的框架比如torchvision，torchtext他们是用于不同问题任务的库，而他们的内部都有自己的dataset。

In [1]:
import torch
from torch import nn
torch.__version__

'2.1.2'

In [2]:
# setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

首先从一个小的数据集开始，比如不是101种而是3种分类的问题，底层原理是一样的但是却可以更清晰的更快捷的进行实验。Let's Experiment!

In [ ]:
import requests
import zipfile
from pathlib import Path

# setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# if the image folder does's exsit, download it
if image_path.is_dir():
    print(f"{image_path} directory exsits.")
else:
    print(f"{image_path} does't exsits, creating one...")
    image_path.mkdir(parent=True, exsit_ok=True)
    # download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https//github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)
    # unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...")
        zip_ref.extractall(image_path)

用大量的时间进行数据准备！查看数据集的结构，将它准备为pytorch可以处理的张量，batch，dataloader。

In [ ]:
# use os.walk()
import os
def walk_through_dir(dir_path):
    """
    Walks through dir_path returning its contents
    Args:
    -----
        dir_path (str or pathlib.Path):target directory

    Returns:
    -----
        A print out of:
        number of subdiretories in dir_path
        number of images files in each subdirectory
        name of each subdirectory
    """
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")

In [ ]:
# setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

可视化总是很重要！
使用pathlib.Path.glob()找到所有的图片文件。

---
`pathlib.Path.glob()`是 `pathlib` 模組提供的一個方法，用於通過樣式匹配返回符合條件的文件或目錄的生成器。

具體來說，`glob()` 方法接受一個模式字符串作為參數，該模式字符串可以包含通配符（例如 `*` 和 `?`），並且它會返回一個生成器，該生成器生成與模式匹配的所有文件或目錄的 `Path` 對象。

以下是一個簡單的示例：

```python
from pathlib import Path

# 在當前目錄中查找所有以 .txt 結尾的文件
for file_path in Path('.').glob('*.txt'):
    print(file_path)
```

上面的例子會列印出當前目錄中所有以 `.txt` 結尾的文件的路徑。 `glob()` 方法的主要用途是方便地遍歷符合特定模式的文件或目錄。

---
返回类名的方法：
`parent` 屬性是 `pathlib.Path` 對象的一個屬性，它返回該路徑對象的父目錄。而 `stem` 屬性則是返回不帶有文件擴展名的文件名部分。

如果你使用 `parent.stem`，則代表取得該路徑的父目錄的文件名部分（不包括擴展名）。

以下是一個簡單的示例：

```python
from pathlib import Path

file_path = Path('/path/to/parent/file.txt')

# 取得父目錄的文件名部分
parent_stem = file_path.parent.stem

print(parent_stem)
```

在這個例子中，`file_path.parent` 將是 `/path/to/parent`，而 `file_path.parent.stem` 將返回 `parent`。這是由於 `parent` 返回的是一個 `Path` 對象，而 `stem` 返回該路徑的文件名部分。

---

打开文件使用`PIL.Image.open()`

In [4]:
# exsample
from pathlib import Path

file_path = Path('/path/to/parent/file.txt')

# 取得父目錄的文件名部分
parent_stem = file_path.parent.stem

print(parent_stem)

parent


In [ ]:
import random
from PIL import Image

# seed seed
random.seed(42)

# 1, get all image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))

# 2, get random image path
random_image_path = random.choice(image_path_list)

# 3, get image class from path name
image_class = random_image_path.parent.stem

# 4, open image
img = Image.open(random_image_path)

# 5, print metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
# can user matplotlib as well, have to convert the image to numpy array
import numpy as np
import matplotlib.pyplot as plt

# turn the image into an array
img_as_array = np.asarray(img)

# plot the image with matplotlib
plt.figure(figsize=(10, 7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape} -> [height, width, color_channels]")
plt.axis(False);

数据转换！要将图片数据加载进pytorch，需要进行张量转换，以及用dataloader将图片打成批次以便后续进行训练。

In [5]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# transform data with torchvision.transfroms
